# Tips and tricks for visualization

This tutorial will show you how to draw closed contours with `folium`.

We will draw isochronous areas computed with the [OAlley API](https://api.oalley.fr).


## Running this example

If you want to run this example to compute your own isochrones, you will need an OAlley API key. Go to [api.oalley.fr](https://api.oalley.fr) and signup. It's free. Each new user receives a key with 1000 requests available.

Paste the key in the code below.

Also, this example requires a version of folium > 0.3. Once you have downloaded this file, Don't forget to hit <kbd>Ctrl</kbd> + <kbd>enter</kbd> to run this.

In [22]:
import folium
import grequests
import polyline
from urllib.parse import urlencode

# This should display a version > 0.3, otherwise PolygonMarker is not supported.
print(folium.__version__)

baseurl = "https://api.oalley.fr/api/AppKeys/"
method = "/isochronous?"

appkey = "YOUR-API-KEY"

0.3.0.dev


## Compute isochrones

In [ ]:
durations = [ 60, 120, 180]
center =  {
    "lat":48.8738,
    "lng":2.295,
    "duration": 0.0
}

urls = [baseurl + appkey + method + urlencode(dict(center,duration=d)) for d in durations]

zones = []

# Called when OAlley returned a zone
def callback(res, **kwargs):
    if(res.status_code != 200): # Most likely, you don't have credits anymore. We give more for free. Contact us at contact@oalley.fr 
        return print(res.json()) 
    body = res.json()
    zone = polyline.decode(body['polyline'])
    zones.append(zone)
    
def exception_handler(request, exception):
    print("Error : %s" % exception)
    
# Prepare all requests
reqs = [grequests.get(url, callback=callback) for url in urls]

# Execute all requests
grequests.map(reqs, exception_handler=exception_handler)

## Display the zones

Play around with the fill and contour colors.

**Note**: The `fit_bounds` method seems broken as of 0.3.0. This is specific to PolygonMarker. The fitted view does not actually fit to the complete size of the polygons. Should be fixed in a near future. 

In [23]:
# Build output map
m = folium.Map(location=[46, 2])

# Contour color supports transparency. Here the last 00 means the contours is fully transparent
contour_color = "#ffffff00" 

# Fill color doesnt support transparency. The last 2 numbers for transparency must be removed.
# Otherwise it will default to black
fill_color    = "#002d5f"

# Draw all computed zones on the map
for points in zones:
    folium.features.PolygonMarker(locations=points, color=contour_color, fill_color=fill_color, weight=10).add_to(m)

# Print the result
m.fit_bounds(m.get_bounds())

# IPython trick to display the map
m

## Next

Go to the [`repository page`](https://github.com/Overdrivr/interactive-geospatial-analysis) to see all other examples and experimentations, test them yourself and contribute if you feel like it.